# Calculating equilibria in game theory

In this notebook, we will consider game theory, with a focus on the two-agent, finite-action case. Two-player simply means there are only two agents facing each other. Finite-action means their decisions only come from a finite set of actions; this is in contrast to games where users may pick values in a continuum, e.g. they choose a number between 0 and 1.

Much of this material is based on the development of Easley and Kleinberg's *Networks, Crowds, and Markets*, which is available for free online.

To remain consistent with the code we will generate, we will use 0-indexing throughout, and also just name things by their index.

So, we have two agents, agents `0` and `1`. We suppose agent `0` has $m$ actions, which are `[0,1,2,...,m-1]` and agent `1` has $n$ actions, which are `[0,1,2,...,n-1]`.

If agent `0` chooses action `i` and agent `1` chooses action `j`, this induces a particular state of the world. We suppose in this case, agent `0` incurs cost `A[i,j]` ($A_{i,j}$ in math notation) and agent `1` incurs cost `B[i,j]` ($B_{i,j}$ in math notation). Informally, we think of agent `0` as wishing to choose `i` to minimize `A[i,j]` and agent `1` as wishing to choose `j` as to minimize `B[i,j]`.

Note that if $A = B$, the goals of agent `0` and agent `1` are perfectly in line: they want the same thing. In essence, the two would try to solve:
$$
\min_{i,j} A_{i,j}
$$
Since the costs are equal, this can also be written:
$$
\min_{i,j} B_{i,j}
$$

As an example, consider:
$$
A = B =
\begin{bmatrix}
0 & 1 \\
2 & 0 \\
\end{bmatrix}
$$
Both agents would hope that the outcome is either `(0,0)` or `(1,1)`.

However, sometimes agents `0` and `1` may not be able to easily coordinate `i` and `j` to choose `(i,j)` that minimizes the cost. For example, if both agents have to pick an action without any prior communication and without seeing the other agent's action, how would they decide what to pick? These questions are generally covered by the term **distributed optimization**. Economists often refer to these games as purely cooperative.

On the exact opposite end of the spectrum, we may have $A = -B$. In words, this means that agent `0` and `1` want the exact opposite outcome to happen. Whatever cost agent `0` incurs, agent `1` incurs the opposite. Intuitively, however happy agent `0` is with the outcome, agent `1` is exactly unhappy in equal magnitude. These games are known as purely adversarial and are called **zero-sum games**.

Usually, we'll deal with games that live somewhere in between: $A$ is neither equal to $B$ or $-B$.

Additionally, note that you will consistently see different conventions. Economists often model agents as **maximizing utility**, whereas engineers often model agents as **minimizing costs**. Be careful to adhere to the correct convention (maximization or minimization) when reading papers or invoking code packages.

## Assumptions when applying this model

Before we go into the analysis of this topic, let's go over when this can model situations in the real world. First, we must assume that every factor in the agents' decision making is accounted for in the cost matrices $A$ and $B$.

For example, if you were trying to model an agent at a restaurant, it would be wrong to model their cost as how much they pay. For two outcomes, "do not tip and feel bad" and "tip", most people would choose to tip their waiter. However, if you model the cost as the total bill at the restaurant, your economic agent would never tip, and you would wind up with a relatively poor model of the world.

Thus, the cost you use when you model these agents must account for all of these factors. There is some 'cost' associated with not tipping, namely the monetary value of "feeling bad". From an economist perspective, this 'cost' is how much an agent is willing to pay to "not feel bad" in that sense.

Second, we must assume that the structure of the game is **common knowledge**. That is, every agent knows the how many agents there are, what actions each agent can take, and the cost each agent will incur for any combination of actions. Furthermore, every agent knows that every other agent knows this information. Furthermore, every agent knows that every other agent knows that every other agent knows this information. And so on, ad infinitum.

Note why this assumption is useful. It means that when I am reasoning about my opponent's strategic decision, I assume they are behaving strategically, and I can model the other agent as a strategic agent in my head. If this were not the case, I may reason about things differently.

For example, suppose I am playing a game of chess with an opponent. I know all the rules of chess, and I assume my opponent knows all the rules of cheese. Furthermore, I assume he knows I know he knows I know that he knows the rules to chess. This sets the framework for the type of reasoning I do.

If, for example, my opponent did not know certain rules, and I knew this, I could take advantage of it. For example, if my opponent does not know about castling, I may make strategic plays around that information.

As another example, suppose my opponent did not know I knew he knew the rules of chess, then he may wish to fake ignorance about the rules of chess to make bad moves. However, if I knew he knew the rules of chess, I would see through this. Analyzing rational behavior in the presence of these *structural uncertainties* is very difficult.

(However, it is worth noting that it is possible to account for asymmetric information in this setting. Suppose I want to buy a house. Another potential buyer gets an appraisal for the house, and makes a bid. I do not know the appraisal value, but I know the bid the potential buyer made. I can reason about the bid and learn something about the appraisal value. The other potential buyer has more information than me, but the structural difference in information is common knowledge to everyone, which still allows for rational reasoning.)

In summary:

1) The costs account for all factors relevant for decision making.

2) The available actions and costs of all players are common knowledge, i.e. it is known to everyone, and everyone knows they know, and everyone knows they know they know, and &c.

## Best response

Given the action `j` of agent `1`, we can define the **best response** of agent `0`. In this case, suppose we know that agent `1` will choose `j`. What is the best decision agent `0` can make?

Well, they would choose action `i` that minimizes `A[i,j]`. 

Implement this in code now.

In [31]:
# load packages and setup variables

import numpy as np

m = 6
n = 4

A = np.array( [  [5,0,3,3],
                 [7,9,3,5],
                 [2,0,3,0],
                 [8,8,4,6],
                 [7,7,8,1],
                 [5,9,8,9]])

B = np.array( [  [ 5,16,19, 5],
                 [15,15,17,18],
                 [ 3,17,19,19],
                 [ 0,14, 7, 0],
                 [ 1, 9, 5,10],
                 [ 3,11,18, 8]])

print('the cost matrix for agent 0 is: \n{}\n'.format(A))
print('the cost matrix for agent 1 is: \n{}'.format(B))

the cost matrix for agent 0 is: 
[[5 0 3 3]
 [7 9 3 5]
 [2 0 3 0]
 [8 8 4 6]
 [7 7 8 1]
 [5 9 8 9]]

the cost matrix for agent 1 is: 
[[ 5 16 19  5]
 [15 15 17 18]
 [ 3 17 19 19]
 [ 0 14  7  0]
 [ 1  9  5 10]
 [ 3 11 18  8]]


# TASK

In [32]:
# given a cost matrix A and the the action j of the other agent, calculate the best responses
# NOTE THAT IF THERE ARE MULTIPLE BEST RESPONSES, YOU ARE EXPECTED TO GIVE ALL OF THEM
# thus, if there is one best response and it is 0, you would return [0]
# if the best responses are 0 and 1, you would return [0,1]
def best_response(A,j):
    
    # TODO
    return np.where(A[:,j] == np.min(A[:,j]))[0].tolist()

print('the cost matrix for agent 0 is: \n{}\n'.format(A))
for j in range(n):
    print('given agent 1 chooses action {}, the best response is {}.'.format(j, best_response(A,j)))

the cost matrix for agent 0 is: 
[[5 0 3 3]
 [7 9 3 5]
 [2 0 3 0]
 [8 8 4 6]
 [7 7 8 1]
 [5 9 8 9]]

given agent 1 chooses action 0, the best response is [2].
given agent 1 chooses action 1, the best response is [0, 2].
given agent 1 chooses action 2, the best response is [0, 1, 2].
given agent 1 chooses action 3, the best response is [2].


We can define things very similarly for agent `1`. If we know agent `0` chooses action `i`, what is the best response for agent `1`?

Note that there is a symmetry to the problem. Without defining a new function, how can we calculate the best response of agent `1` when given the action `i` of agent `0`?

# TASK

In [33]:
print('the cost matrix for agent 1 is: \n{}\n'.format(B))
for i in range(m):
    
    # TODO
    br = best_response(B.T,i)
    
    print('given agent 0 chooses action {}, the best response is {}.'.format(i,br))

the cost matrix for agent 1 is: 
[[ 5 16 19  5]
 [15 15 17 18]
 [ 3 17 19 19]
 [ 0 14  7  0]
 [ 1  9  5 10]
 [ 3 11 18  8]]

given agent 0 chooses action 0, the best response is [0, 3].
given agent 0 chooses action 1, the best response is [0, 1].
given agent 0 chooses action 2, the best response is [0].
given agent 0 chooses action 3, the best response is [0, 3].
given agent 0 chooses action 4, the best response is [0].
given agent 0 chooses action 5, the best response is [0].


## Dominated equilibrium

When we talk about games, we often want to talk about **equilibria**. Intuitively, equilibria are stable (in some sense of the word) points in the game; the strategic interactions of all the agents balance out in some sense. As we go over the types of equilibria, we will see formal definitions of this 'stability' or 'balance'.

The first type of equilibrium we will go over is the dominant strategy equilibrium.

For agent `0`, a **dominant strategy** `i` is one such that for **every action $j$** of agent `1`, `i` is the best response. That is, regardless of what agent `1` does, it is in agent `0`'s best interest to play `i`.

Note that in this sense, there's actually not that much room for strategy. The agent doesn't need to think about anything, and should simply just play their dominant strategy `i`. It's always the best response. It's always the best thing to do.

# TASK

In [34]:
# given a cost matrix A, find the dominant strategies of agent i
# if there isn't one, return []
def find_dominant_strategy(A):
    
    (m,n) = A.shape
    
    # TODO
    best_moves = []
    for j in range(n):
        best_moves.append(best_response(A,j))
    if len(best_moves) > 0:
        best_move = best_moves[0]
        for j in range(1,n):
            best_move = np.intersect1d(best_move, best_moves[j])
            if best_move.size == 0:
                return []
        return best_move
    else:
        return []
    
print('the dominant strategies for agent 0 are: {}'.format(find_dominant_strategy(A)))

the dominant strategies for agent 0 are: [2]


As before, there is symmetry, so how can we make this same function find the dominant strategies for agent `1`?

# TASK

In [35]:
# TODO
dominant_strategies = find_dominant_strategy(B.T)
# dominant_strategies = []
print('the dominant strategies for agent 1 are: {}'.format(dominant_strategies))

the dominant strategies for agent 1 are: [0]


Note that there may not always exist a dominant strategy for each agent.

However, if every agent has a dominant strategy, we can define a **dominant strategy equilibrium** as the dominant strategies for each agent.

So, consider the prisoner's dilemma:
$$
A =
\begin{bmatrix}
-3 & -1 \\
-4 & -2
\end{bmatrix}
\quad
B =
\begin{bmatrix}
-3 & -4 \\
-1 & -2
\end{bmatrix}
$$

The dominant strategy equilibrium here would be `[i = 0, j = 0]`.

Dominant strategy equilibria are very 'stable'. There are many formalizations of the notion of stability, which are beyond the scope of this course. However, you can note that if agent `1` chooses to do some sub-optimal actions, agent `0` will still stick to his action. If agent `1` plays randomly, agent `0` will stick to his action. If agent `0` messes up and chooses the wrong action on one turn, he's very likely to return to the dominant strategy action.

# TASK

In [ ]:
# given the cost matrices A and B, find the dominant strategy equilibria
def find_dominant_strategy_equilibria(A,B):
    
    # TODO
    
    return [[0,0]]

print('the dominant strategy equilibria are: {}'.format(find_dominant_strategy_equilibria(A,B)))

## Pure Nash equilibria

A set of actions `[i,j]` is a **pure Nash equilibrium** if `i` is a best response to `j` for agent `0` and `j` is a best response to `i` for agent `1`.

In words, this means that no player has an incentive to deviate.

Note that every dominant strategy equilibrium is a pure Nash equilibrium. However, not every pure Nash equilibrium is a dominant strategy equilibrium.

In a certain sense, a pure Nash equilibrium is 'stable': no player unilaterally has any incentive to change their actions. However, it is also quite brittle: if one player chooses a different action, the other players might suddenly have different best responses they would rather play. They may no longer want to choose their action in the pure Nash equilibrium.

Consider the game of Bach versus Stravinsky:
$$
A =
\begin{bmatrix}
0 & 2 \\
2 & 1
\end{bmatrix}
\quad
B =
\begin{bmatrix}
1 & 2 \\
2 & 0
\end{bmatrix}
$$
What are the pure Nash equilibria? We can see that games can have multiple pure Nash equilibria.

Similarly, consider the matching pennies game:
$$
A =
\begin{bmatrix}
1 & -1 \\
-1 & 1 \\
\end{bmatrix}
\quad
B =
\begin{bmatrix}
-1 & 1 \\
1 & -1
\end{bmatrix}
$$
This game has no pure Nash equilibria.

# TASK

In [ ]:
def find_pure_Nash_equilibria(A,B):
    
    (m,n) = A.shape
    
    # TODO
                
    return [[0,0]]

print('the pure Nash equilibria are: {}'.format(find_pure_Nash_equilibria(A,B)))

A2 = np.array( [ [ 0, 0, 4, 5],
                 [ 6, 8,17,15],
                 [ 4, 9,10, 1],
                 [ 1, 7, 0, 3],
                 [ 6,11,14,18],
                 [ 0,14, 3,12]])

B2 = np.array( [ [10, 3, 4, 6],
                 [ 4,15, 3,12],
                 [ 4, 8,14,15],
                 [ 3,15, 2,16],
                 [17, 5, 9, 3],
                 [ 0, 5, 0,17]])

print('the pure Nash equilibria are: {}'.format(find_pure_Nash_equilibria(A2,B2)))

## Mixed Nash equilibria

We now introduce the potential for randomization. Namely, each agent independently draws their action from a probability distribution of their choice. 

So, suppose agent `1` draws their action from a probability distribution $p_1$, where $p_1(j)$ is the probability he will draw action $j$. The best response of agent `0` to $p_1$ is defined as the probability distributions $p_0$ such that:
\begin{align}
\min_{p_0}        ~& \sum_{i = 0}^{m-1} \sum_{j = 0}^{n-1} p_0(i) p_1(j) A_{i,j} \\
\text{subject to} ~& p_0(i) \geq 0 \text{ for } i = 0,\dots,m-1 \\
                   & \sum_{i =0}^{m-1} p_0(i) = 1
\end{align}
Here, the constraints require that $p_0$ is a valid probability distribution.

Suppose $p_0$ is a best response. Then any probability distribution $p_0'$ such that $p_0(i)$ implies $p_0'(i) = 0$ is also a best response.

In words, if $p_0$ is a best response, than any probability distribution that distributes is probabilities on the same actions as $p_0$ is also a best response. Can you see why this is the case?

Now, your task is not to calculate the best response, but find **which actions can have positive weight in the best response**. (The elements with positive weight are often called the **support** of the probability distribution.)

# TASK

In [ ]:
# find the actions that can be in the mixed best response to p
def mixed_best_response_support(A,p):
    
    # TODO
    
    return [0]

print(mixed_best_response_support(A,[1/6, 3/6, 2/6, 0]))

A3 = np.array( [ [ 1,-1],
                 [-1, 1]])

print(mixed_best_response_support(A3,[1/2,1/2]))

A mixed Nash equilibria is a set of probability distributions $(p_0,p_1)$ such that $p_0$ is a mixed best response to $p_1$ and similarly $p_1$ to $p_0$.

Now, Nash's theorem is that every finite game has a mixed Nash equilibrium. The proof is non-constructive, which means that the proof doesn't actually explicitly calculate the mixed Nash equilibrium. It uses more indirect proof methods, which demonstrate existence, but don't give us a means to look at the object that exists.

Oftentimes, mixed Nash equilibria will be referred to as **Nash equilibria** for short. Thus, you may hear people say that a Nash equilibrium always exists. This may confuse you, as we have previously shown games where no pure Nash equilibria exists. Without further qualification, people are usually referring to mixed Nash equilibria in these situations.

It is more complicated to calculate mixed Nash equilibria, so you will not be asked to in this notebook.

## Equilibrium selection

We know that a mixed Nash always exists, but when there are multiple equilium, it's difficult to come up with an argument as to "what will happen". This is the problem of **equilibrium selection** and an active research topic.

Consider the coordination game:
$$
A =
\begin{bmatrix}
0 & 4 \\
4 & 2 
\end{bmatrix}
\quad
B =
\begin{bmatrix}
0 & 4 \\
4 & 2
\end{bmatrix}
$$
There are two pure Nash equilibria: `[0,0]` and `[1,1]`. However, we can see in a certain sense `[0,0]` is better: every player is better off than in the equilibrium `[1,1]`. This is often called a **focal point**.

One solution is **correlated equilibria**. We won't cover the formal details, but consider the traffic light problem. 

I am at an intersection with another person. I can choose to go, or wait. The other driver has the same actions available. My most desired outcome is that I go, and the other driver waits. I want to get to my destination sooner rather than later. My least desired outcome is that we both go, and crash into each other.

With go = `0` and wait = `1`, we can think of the game as:
$$
A =
\begin{bmatrix}
10 & 0 \\
1  & 1
\end{bmatrix}
\quad
B =
\begin{bmatrix}
10 & 1 \\
0  & 1
\end{bmatrix}
$$

What do we do in the real world? We installed a traffic light. It randomly decides to give one of us a red light and the other a green light. Even though I wish I were the one given a green light, once the lights are assigned, I obey.

Suppose a coordinator draws actions for everyone from some joint probability distribution and tells everyone what to do. Furthermore, suppose every agent knows the joint probability distribution that the coordinator uses. If everyone chooses to listen to the coordinator, then this is a correlated equilibrium.

Note that, formally, each person is only told what to do, not the joint action chosen. So, in the traffic example, I only see my light signal, and cannot see the light signal of the other driver.

Similarly, note that this is not always 'fair'. For example, even if the other driver gets a green light 2/3 of the time, I will still obey the traffic light.

## Optimality of equilibria

Finally, the last topic we'll cover is two criteria by which equilibria are considered 'optimal'. This type of reasoning is common in equilibria selection. For example, if we have 5 equilibria but one of them is 'optimal' in some sense, then this is the one that we will gravitate towards.

A set of actions `[i,j]` is **Pareto optimal** if there are no actions `[i2,j2]` such that `A[i2,j2] <= A[i,j]` and `B[i2,j2] <= B[i,j]`, with one of the two holding **strictly**. That is, there is no one to give someone strictly lower cost and keep everyone else's cost at most as low as it was before. Improving one person's cost **must** cause someone else's cost to be worse. Consider the coordination game above. Is one of the two equilibria Pareto optimal?

Second, a set of actions `[i,j]` is **socially optimal** if it minimizes `A[i,j] + B[i,j]`. If we sum every person's cost together, which equilibria minimizes the total? Note that this requires the costs to be 'comparable', in the sense that they have the same units and can be added. This is a common problem in economics, as how do you compare the happiness of Abe to the happiness of Susan? This is a complicated topic I normally lecture on for a few weeks, but falls outside the scope of this class. 

A short summary is that oftentimes we don't directly look at happiness, but 'willingness to pay'. So, I can't say how happy a new car makes you, and I have no way to estimate that or units to quantify that in. However, I can estimate how much money you are willing to spend to buy this new car. If you are willing to pay X dollars, I can conclude that you value having the car at least more than having X dollars.

In mechanism design, this type of assumption is normally wrapped up in the **quasilinearity** assumption, where the utility of outcomes can be summed with payments, and thus is implicitly in units of dollars.